# 1_scrape_core_preview — (aggiornato da `3_run_all.ipynb`)

Questo notebook esegue **Parte 1**:
- Scraping di decklist e matchups
- Consolidamento/alias
- Costruzione matrici W/L/T/WR e `n_dir`
- Filtro NaN e **score_latest/filtered_wr/n_dir**

> Fonte di verità: il notebook 3. Le celle seguenti sono **estratte e allineate**.

In [ ]:
import logging

# 1) azzera gli handler duplicati di Jupyter ed imposta un’unica configurazione
logging.basicConfig(
    level=logging.INFO,           # livello generale del progetto
    format="%(levelname)s:%(name)s:%(message)s",
    force=True                    # 👈 IMPORTANTISSIMO in Jupyter per evitare duplicazioni
)

# 2) silenzia SOLO il logger di rete
logging.getLogger("ptcgp.net").setLevel(logging.WARNING)

# 3) opzionale: riduci rumore di webdriver-manager / selenium
logging.getLogger("WDM").setLevel(logging.WARNING)
logging.getLogger("selenium").setLevel(logging.WARNING)

# 4) se vuoi vedere più diagnostica del tuo codice ma non il traffico rete:
logging.getLogger("ptcgp").setLevel(logging.DEBUG)   # tuo codice
logging.getLogger("utils.io").setLevel(logging.INFO) # lascia i "CSV aggiornato" se ti servono


In [ ]:
# Setup base e import dei moduli
from pathlib import Path
import logging
import pandas as pd

from utils.io import init_paths, _dest, write_csv_versioned, save_plot_timestamped
from scraper.decklist import scrape_decklist_html, parse_decklist_table, filter_top_meta, LIMITLESS_DECKS_URL
from scraper.matchups import to_matchup_url, scrape_matchups
from scraper.session import make_session
from core.normalize import load_alias_map, build_alias_index
from core.consolidate import maxN_flat, apply_alias_and_aggregate, build_score_table_filtered
from core.matrices import topmeta_post_alias, build_matrices, n_dir_from_WL
from core.nan_filter import filter_wr_nan_iterative

# ⚠️ NON richiamare basicConfig: già impostato in cella 1 con force=True
log = logging.getLogger("ptcgp")

BASE = Path.cwd()
paths = init_paths(BASE)
paths


In [ ]:
# Garantisci file di configurazione minimi (alias_map.json e config.yaml) e carica CFG
cfg_dir = BASE / "config"
cfg_dir.mkdir(parents=True, exist_ok=True)

alias_json = cfg_dir / "alias_map.json"
if not alias_json.exists():
    alias_json.write_text("{}\n", encoding="utf-8")
    log.warning("[init] creato config/alias_map.json vuoto — nessun alias verrà applicato finché non lo compili.")

yaml_file = cfg_dir / "config.yaml"
if not yaml_file.exists():
    yaml_file.write_text("# config placeholder\n", encoding="utf-8")
    log.info("[init] creato config/config.yaml placeholder")

# Lettura della config (come facevi prima)
import yaml  # se non fosse installato, aggiungi 'pyyaml' al venv/requirements
with open(yaml_file, "r", encoding="utf-8") as f:
    CFG = yaml.safe_load(f) or {}
log.info("Config loaded from %s", yaml_file)

# (facoltativo) Allinea livelli di log alla config
lvl = (CFG.get("logging", {}).get("level", "INFO") or "INFO").upper()
logging.getLogger("ptcgp").setLevel(getattr(logging, lvl, logging.INFO))
logging.getLogger("utils.io").setLevel(logging.INFO)


## D1 — Scraping decklist e matchups

- Usa Selenium per la pagina **Decks** (cache 12h lato HTML).
- Converte in tabella e filtra il **Top-meta (80%)**.
- Costruisce le URL `/matchups` e scarica tutte le pagine dei deck top.
- Scrive:
  - `outputs/Decklists/raw/decklist_raw_*latest.csv`
  - `outputs/Decklists/top_meta/top_meta_decklist_*latest.csv`
  - `outputs/MatchupData/raw/matchup_raw_*latest.csv`

In [ ]:
# Esecuzione D1 — scraping + salvataggi raw (parametri presi da CFG)

# --- Parametri da config.yaml ---
scr_cfg   = CFG.get("scraping", {}) if "CFG" in globals() else {}
top_cfg   = CFG.get("top_meta", {}) if "CFG" in globals() else {}

DECKS_URL    = scr_cfg.get("decks_url", LIMITLESS_DECKS_URL)
TTL_MIN      = int(scr_cfg.get("cache_ttl_min", 720))
FORCE_REFRESH= bool(scr_cfg.get("force_refresh", False))
HEADLESS     = bool((scr_cfg.get("selenium") or {}).get("headless", True))
RATE_LIMIT   = float(scr_cfg.get("request_delay_sec", 5.0))
TOP_THRESH   = float(top_cfg.get("threshold_pct", 80.0))

log.info("[scrape] decks_url=%s | ttl_min=%s | headless=%s | force_refresh=%s | rate_limit=%.2fs | top_thresh=%.1f%%",
         DECKS_URL, TTL_MIN, HEADLESS, FORCE_REFRESH, RATE_LIMIT, TOP_THRESH)

# --- Decklist page ---
html, from_cache = scrape_decklist_html(
    DECKS_URL,
    cache_dir=paths.cache, ttl_minutes=TTL_MIN,
    force_refresh=FORCE_REFRESH, headless=HEADLESS
)
df_decklist = parse_decklist_table(html)

out1 = write_csv_versioned(
    df_decklist.reset_index(),
    _dest(paths, "decklist_raw"),
    "decklist_raw",
    changed=(not from_cache),
    index=False
)
log.info("Decklist rows=%d | saved=%s", len(df_decklist), out1)

# --- Top-meta + URL matchups ---
df_top = filter_top_meta(df_decklist, threshold_pct=TOP_THRESH).copy()
df_top["Matchup URL"] = df_top["URL"].map(to_matchup_url)

out2 = write_csv_versioned(
    df_top,
    _dest(paths, "top_meta_decklist"),
    "top_meta_decklist",
    changed=(not from_cache),
    index=False
)
log.info("Top-meta rows=%d | saved=%s", len(df_top), out2)

# --- Scarica matchups per tutti i deck top ---
urls = [(r["Deck"], r["Matchup URL"]) for _, r in df_top[["Deck","Matchup URL"]].dropna().iterrows()]
if not urls:
    raise RuntimeError("Nessun URL matchup trovato dal top-meta (controlla la decklist e la colonna 'URL').")

sess = make_session()
df_raw, total, cache_hits = scrape_matchups(
    urls,
    session=sess, cache_dir=paths.cache,
    ttl_minutes=TTL_MIN, force_refresh=FORCE_REFRESH,
    rate_limit_seconds=RATE_LIMIT,
    progress=True,
    pbar_desc=f"Matchups Top {len(urls)}"
)
sess.close()

# --- Hardening: colonne minime ---
_required = {"Deck A","Deck B","W","L","T"}
missing = _required - set(df_raw.columns)
if missing:
    raise KeyError(f"[matchup_raw] mancano colonne richieste: {missing}")

out3 = write_csv_versioned(
    df_raw,
    paths.outputs / "MatchupData" / "raw",
    "matchup_raw",
    changed=(cache_hits < total) or FORCE_REFRESH,
    index=False
)
log.info("Matchup pages=%d | cache hits=%d | rows=%d | saved=%s", total, cache_hits, len(df_raw), out3)


## D2/D3 — Consolidamento, alias, matrici e filtro NaN

Scrive i contratti:
- `outputs/MatchupData/flat/score_*latest.csv` *(flat aggregata post-alias con W/L/T/N & WR_dir)*
- `outputs/Matrices/winrate/filtered_wr_*latest.csv` *(WR filtrata, diagonale NaN)*
- `outputs/Matrices/volumes/n_dir_*latest.csv` *(N_dir = W+L post-aggregazione)*

In [ ]:
# Carica l'ultima raw
raw_dir = paths.outputs / "MatchupData" / "raw"
raw_latest = max(raw_dir.glob("matchup_raw_*latest.csv"), default=None)
if raw_latest is None:
    raise SystemExit("matchup_raw_latest.csv non trovato: esegui D1")
df_raw = pd.read_csv(raw_latest)

# Consolidamento max-N per (A,B)
df_flat = maxN_flat(df_raw)

# Alias (da config)
alias_cfg    = (CFG.get("alias") or {}) if "CFG" in globals() else {}
APPLY_ALIASES = bool(alias_cfg.get("apply", True))
alias_path    = BASE / alias_cfg.get("file", "config/alias_map.json")

alias_map   = load_alias_map(alias_path) if APPLY_ALIASES else {}
alias_index = build_alias_index(alias_map) if APPLY_ALIASES else {}

# Apply alias + aggregazione (mirror esclusi) + Winrate direzionale
df_flat_alias = apply_alias_and_aggregate(df_flat, alias_index if APPLY_ALIASES else {})

# ⛳️ Salva SOLO la versione pre-filtro come audit (NON deve chiamarsi 'score')
flat_dir = _dest(paths, "matchup_score_table")
write_csv_versioned(df_flat_alias, flat_dir, "score_pre_filter", changed=True, index=False)

# Asse Top-meta post-alias
top_dir    = paths.outputs / "Decklists" / "top_meta"
top_latest = max(top_dir.glob("top_meta_decklist_*latest.csv"), default=None)
if top_latest is None:
    raise SystemExit("top_meta_decklist_latest.csv non trovato: esegui D1")
_df_top     = pd.read_csv(top_latest)
df_top_alias= topmeta_post_alias(_df_top, alias_index if APPLY_ALIASES else {})
axis        = df_top_alias["Deck"].tolist()

# Matrici W/L/T + WR (mode exclude) + n_dir
W, L, T, WR = build_matrices(df_flat_alias, axis, mode="exclude", mirror=None)
N_DIR = n_dir_from_WL(W, L)

# Filtro NaN iterativo (parametri da config)
nan_cfg = (CFG.get("nan_filter") or {}) if "CFG" in globals() else {}
MAX_NAN_RATIO   = float(nan_cfg.get("max_nan_ratio", 0.15))
MIN_NAN_ALLOWED = int(nan_cfg.get("min_nan_allowed", 1))
USE_CEIL        = bool(nan_cfg.get("use_ceil", False))

filtered_wr, dropped = filter_wr_nan_iterative(
    WR,
    max_nan_ratio=MAX_NAN_RATIO,
    min_nan_allowed=MIN_NAN_ALLOWED,
    use_ceil=USE_CEIL
)
kept    = filtered_wr.index.tolist()
N_DIR_f = N_DIR.loc[kept, kept]

# ✅ Costruisci la score table **post-filtro** sull'asse kept (contratto finale)
score_filtered = build_score_table_filtered(
    df_flat_alias=df_flat_alias,
    kept_axis=kept,
    round_wr=2,
    legacy_winrate_alias=True  # crea anche 'Winrate' come alias di 'WR_dir'
)

# Scrivi i contratti (winrate/volumi) + score_latest filtrata
win_dir = _dest(paths, "filtered_wr")
vol_dir = _dest(paths, "n_dir")
write_csv_versioned(filtered_wr, win_dir, "filtered_wr", changed=True, index=True)
write_csv_versioned(N_DIR_f,   vol_dir, "n_dir",       changed=True, index=True)

# 👉 'score_latest.csv' ORA è la versione filtrata & simmetrizzata
out_score = write_csv_versioned(score_filtered, flat_dir, "score", changed=True, index=False)

print("D2/D3 completati — righe flat(pre):", len(df_flat_alias),
      "| WR shape:", filtered_wr.shape,
      "| kept:", len(kept),
      "| score_latest:", out_score)


## Validazione rapida (WR e n_dir)

Controlla che:
- `filtered_wr ≈ 100·W/(W+L)` (entro 0.10 pp)
- `n_dir == W+L` (off‑diag) e simmetrico A↔B
- `WR(A,B) + WR(B,A) ≈ 100` (entro 0.20 pp)

In [ ]:
# ✅ VALIDAZIONE SCORE POST-FILTRO — per-riga calcolato con T0 (asse iniziale del filtro)

from pathlib import Path
import numpy as np
import pandas as pd
from math import floor, ceil
from utils.io import init_paths

EPS_WR  = 0.10  # tolleranza WR (pp)
EPS_SUM = 0.20  # tolleranza per WR(A,B)+WR(B,A)≈100 (pp)

# --- Carica contratti ---
paths    = init_paths(Path.cwd())
df_score = pd.read_csv(paths.outputs/"MatchupData"/"flat"/"score_latest.csv")
df_wr    = pd.read_csv(paths.outputs/"Matrices"/"winrate"/"filtered_wr_latest.csv", index_col=0)
df_n     = pd.read_csv(paths.outputs/"Matrices"/"volumes"/"n_dir_latest.csv",   index_col=0)

# --- Recupera CFG (già in RAM) o rileggi config.yaml ---
try:
    CFG
except NameError:
    import yaml
    with open(Path.cwd()/"config"/"config.yaml", "r", encoding="utf-8") as f:
        CFG = yaml.safe_load(f) or {}

nan_cfg = (CFG.get("nan_filter") or {})
MAX_NAN_RATIO   = float(nan_cfg.get("max_nan_ratio", 0.15))
MIN_NAN_ALLOWED = int(nan_cfg.get("min_nan_allowed", 1))
USE_CEIL        = bool(nan_cfg.get("use_ceil", False))

# --- Asse coerente (finale, post-filtro) ---
wr_loaded = df_wr.copy()
wr_loaded.index   = wr_loaded.index.astype(str).str.strip()
wr_loaded.columns = wr_loaded.columns.astype(str).str.strip()
axis = wr_loaded.index.tolist()
T = len(axis)  # numero di deck tenuti (kept)

# --- Normalizza score & coercizza contatori ---
def _coerce_counts(df, cols=("W","L","T","N")):
    out = df.copy()
    for c in cols:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce").fillna(0).astype("Int64")
    return out

req_cols = {"Deck A","Deck B","W","L","T","N","WR_dir"}
missing = req_cols - set(df_score.columns)
if missing:
    raise KeyError(f"[score_latest] mancano colonne: {missing}")

score = _coerce_counts(df_score)
score["Deck A"] = score["Deck A"].astype(str).str.strip()
score["Deck B"] = score["Deck B"].astype(str).str.strip()
score = score[score["Deck A"].isin(axis) & score["Deck B"].isin(axis)].copy()

# --- Copertura off-diag coerente col filtro ---
off_mask        = ~pd.DataFrame(np.eye(T, dtype=bool), index=axis, columns=axis)
total_offdiag   = T*(T-1)
defined_offdiag = int((wr_loaded.notna() & off_mask).sum().sum())
missing_offdiag = total_offdiag - defined_offdiag
missing_rate    = (missing_offdiag / total_offdiag) if total_offdiag else 0.0

rows = len(score)
rows_match_defined = (rows == defined_offdiag)      # lo score copre esattamente le celle definite
no_overfill        = (rows <= defined_offdiag)
mirrors_ok         = (score["Deck A"] != score["Deck B"]).all()
subset_ok          = set(score["Deck A"]).issubset(set(axis)) and set(score["Deck B"]).issubset(set(axis))

# --- Calcola allowed_per_row usando T0 (asse iniziale del filtro: top-meta post-alias) ---
from core.normalize import load_alias_map, build_alias_index
from core.matrices import topmeta_post_alias

alias_cfg   = (CFG.get("alias") or {})
apply_alias = bool(alias_cfg.get("apply", True))
alias_path  = Path.cwd() / alias_cfg.get("file", "config/alias_map.json")

top_latest = max((paths.outputs/"Decklists"/"top_meta").glob("top_meta_decklist_*latest.csv"), default=None)
if top_latest is None:
    raise SystemExit("top_meta_decklist_latest.csv non trovato")

_df_top   = pd.read_csv(top_latest)
alias_idx = build_alias_index(load_alias_map(alias_path)) if apply_alias else {}
axis0     = topmeta_post_alias(_df_top, alias_idx)["Deck"].tolist()
T0        = len(axis0)

allowed_per_row = max(
    MIN_NAN_ALLOWED,
    (ceil if USE_CEIL else floor)(MAX_NAN_RATIO * (T0 - 1))
)
row_missing = (wr_loaded.isna() & off_mask).sum(axis=1).astype(int)
row_ok = bool((row_missing <= allowed_per_row).all())
print(f"[POLICY] T0={T0} -> allowed_per_row={allowed_per_row} (MAX_NAN_RATIO={MAX_NAN_RATIO}, "
      f"min={MIN_NAN_ALLOWED}, {'ceil' if USE_CEIL else 'floor'})")

print(f"[BASIC] T_final={T} | off-diag tot={total_offdiag} | definite={defined_offdiag} | "
      f"righe score={rows} | missing={missing_offdiag} ({missing_rate:.1%}) | "
      f"rows_match_defined={rows_match_defined}")

# --- Entrambe le direzioni per le coppie presenti nello score ---
pair_counts = score.apply(lambda r: tuple(sorted((r["Deck A"], r["Deck B"]))), axis=1).value_counts()
both_dirs_ok = bool((pair_counts == 2).all())
if not both_dirs_ok:
    print("[DIRS] coppie nello score senza entrambe le direzioni (prime 10):")
    print(pair_counts[pair_counts != 2].head(10))

# --- Coerenza riga-per-riga ---
n_bad_rows = int((score["N"] != (score["W"] + score["L"] + score["T"]).astype("Int64")).sum())
den_row    = (score["W"] + score["L"]).astype("Int64")
wr_row_calc = np.where(den_row > 0, 100.0 * score["W"].astype(float) / den_row.astype(float), np.nan)
wr_row_bad  = int((np.abs(score["WR_dir"].astype(float) - np.round(wr_row_calc, 2)) > EPS_WR).sum())
winrate_bad = int((score.get("Winrate", score["WR_dir"]).round(2) != score["WR_dir"].round(2)).sum())
print(f"[ROW] N≠W+L+T: {n_bad_rows} | |WR-Formula|>{EPS_WR:.2f}pp: {wr_row_bad} | Winrate≠WR_dir: {winrate_bad}")

# --- Matrici da score e confronti avanzati ---
W = (score.pivot_table(index="Deck A", columns="Deck B", values="W", aggfunc="sum", fill_value=0)
           .reindex(index=axis, columns=axis).astype(float))
L = (score.pivot_table(index="Deck A", columns="Deck B", values="L", aggfunc="sum", fill_value=0)
           .reindex(index=axis, columns=axis).astype(float))
Tmat = (score.pivot_table(index="Deck A", columns="Deck B", values="T", aggfunc="sum", fill_value=0)
            .reindex(index=axis, columns=axis).astype(float))

den = W + L
wr_calc = (W * 100.0 / den).where(den > 0).round(2)

wr_loaded_cmp = wr_loaded.copy()
np.fill_diagonal(wr_calc.values,       np.nan)
np.fill_diagonal(wr_loaded_cmp.values, np.nan)

both = wr_loaded_cmp.notna() & wr_calc.notna()
n_cells = int(both.sum().sum())
n_bad   = int(((wr_loaded_cmp.where(both) - wr_calc.where(both)).abs() > EPS_WR).sum().sum())
print(f"[WR=matrix] celle confrontate={n_cells} | fuori>{EPS_WR:.2f}pp: {n_bad}")

wr_sum = (wr_loaded + wr_loaded.T)
np.fill_diagonal(wr_sum.values, np.nan)
bad_100 = int(((wr_sum - 100.0).abs() > EPS_SUM).sum().sum())
print(f"[SUM=100] celle fuori>{EPS_SUM:.2f}pp: {bad_100}")

n_loaded = df_n.reindex(index=axis, columns=axis).astype(float)
n_calc   = (W + L).astype(float)
np.fill_diagonal(n_loaded.values, np.nan)
np.fill_diagonal(n_calc.values,   np.nan)

cmp_off  = (n_loaded.fillna(-1) != n_calc.fillna(-1)) & off_mask
mm_off   = int(cmp_off.sum().sum())
asym_off = int(((n_loaded - n_loaded.T).where(off_mask).fillna(0) != 0).sum().sum())
W_vs_Lt_bad = int(((W - L.T).where(off_mask).fillna(0) != 0).sum().sum())
T_sym_bad   = int(((Tmat - Tmat.T).where(off_mask).fillna(0) != 0).sum().sum())
print(f"[N_DIR] mismatch off: {mm_off} | asimmetrie off: {asym_off}")
print(f"[SYM]   W≠L^T off: {W_vs_Lt_bad} | T≠T^T off: {T_sym_bad}")

# --- Esito (coerente con la policy del filtro calcolata su T0) ---
ok = (
    mirrors_ok and subset_ok and no_overfill and rows_match_defined and both_dirs_ok
    and (missing_rate <= MAX_NAN_RATIO) and row_ok                # per-riga calcolato da T0
    and n_bad_rows == 0 and wr_row_bad == 0 and winrate_bad == 0
    and n_bad == 0 and bad_100 == 0 and mm_off == 0 and asym_off == 0
    and W_vs_Lt_bad == 0 and T_sym_bad == 0
)
print("\n=== VALIDATION:", "PASS ✅" if ok else "CHECK ⚠️", "===")
